<a href="https://colab.research.google.com/github/MarkusShipley/big-data-challenge/blob/main/bigdatachallenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [61]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:8 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:9 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Fetched 252 kB in 2s (106 kB/s)
Reading package lis

In [62]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("big-data-challenge").getOrCreate()

In [63]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Major_Appliances_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
appliance_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Major_Appliances_v1_00.tsv.gz"), sep="\t", header=True)
appliance_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   16199106|R203HPW78Z7N4K|B0067WNSZY|     633038551|FGGF3032MW Galler...|Major Appliances|          5|            0|          0|   N|                Y|If you need a new...|What a great stov...| 2015-08-31|
|         US|   16374060|R2EAIGVLEALSP3|B002QSXK60|     811766671|Best Hand Clothes...|Major Appliances|          5|    

In [64]:
# Size of Data
# Row Count
appliance_df.count()


96901

In [65]:
# Clean up DataFrames to match tables
from pyspark.sql.functions import to_date
# Review DataFrame
review_id_df = appliance_df.select(["review_id", "customer_id", "product_id", "product_parent", to_date("review_date", 'yyyy-MM-dd').alias("review_date")])
review_id_df.show()


+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R203HPW78Z7N4K|   16199106|B0067WNSZY|     633038551| 2015-08-31|
|R2EAIGVLEALSP3|   16374060|B002QSXK60|     811766671| 2015-08-31|
|R1K1CD73HHLILA|   15322085|B00EC452R6|     345562728| 2015-08-31|
|R2KZBMOFRMYOPO|   32004835|B00MVVIF2G|     563052763| 2015-08-31|
| R6BIZOZY6UD01|   25414497|B00IY7BNUW|     874236579| 2015-08-31|
|R1MCXZFNF8E7Y0|   36311751|B0033X29CI|     294467812| 2015-08-31|
|R3EMB3E3ODR6BW|   30920961|B005R597HA|     183784715| 2015-08-31|
| RJTONVTTOPJ5S|   52491265|B00MO6V8Y0|     960251524| 2015-08-31|
|R21U5QZ2CQECUM|   48166169|B00HT39QDI|     992475314| 2015-08-31|
| RL2BBC51H89DH|   50394924|B00LESFZ52|       1641606| 2015-08-31|
|R3RNEPHF3WIRSZ|    3915552|B0149IJVPI|     838108342| 2015-08-31|
|R38DNT9KML2PF3|   17068589|B002HT0958|     387104338| 2015-08

In [66]:
# Products DataFrame
prod_df = appliance_df.select(["product_id", "product_title"]).drop_duplicates()
prod_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00RE645WC|WindMax 23" Stain...|
|B008SCVL6E|G.E. Microwave Gl...|
|B0081LV860|Samsung WF405ATPA...|
|B00KC812WK|2-pack OnePurify ...|
|B00MEPNYRK|GE PSB48YSHSS Pro...|
|B000JONLMQ|KitchenAid 18" TR...|
|B0073M7GNC|Whirlpool WOS92EC...|
|B00BIWR3IQ|GE Profile PWE23K...|
|B00O3XF1RC|MobileWasher Kit ...|
|B00JDB6P3I|Bosch NGM5055UC 5...|
|B0011YJE7Y|GE PHP960DMBB Pro...|
|B00J4EBYBM|Bosch HBL5651UC 5...|
|B0056HJ07Q|Frigidaire FHPC36...|
|B00FGWW82A|Frigidaire FFFS51...|
|B004UM5Y32|None Ers30t10074 ...|
|B00MANTPJM|GE Profile PFE28R...|
|B006L8PW1C|Fisher Paykel DD2...|
|B00T9WOH5E|Lg Lt600p Compati...|
|B001DHRJSU|Whirlpool Part Nu...|
|B0050KKN8E|Whirlpool Part Nu...|
+----------+--------------------+
only showing top 20 rows



In [67]:
# Reviews DataFrame
reviews_df = appliance_df.select(["review_id", "review_headline", "review_body"])
reviews_df.show()

+--------------+--------------------+--------------------+
|     review_id|     review_headline|         review_body|
+--------------+--------------------+--------------------+
|R203HPW78Z7N4K|If you need a new...|What a great stov...|
|R2EAIGVLEALSP3|          Five Stars|        worked great|
|R1K1CD73HHLILA|       Fast Shipping|Part exactly what...|
|R2KZBMOFRMYOPO|          Five Stars|Love my refrigera...|
| R6BIZOZY6UD01|          Five Stars|No more running t...|
|R1MCXZFNF8E7Y0|       Piece of Junk|It would not cool...|
|R3EMB3E3ODR6BW|Works awesome for...|Works awesome for...|
| RJTONVTTOPJ5S|          Five Stars|exactly what I wa...|
|R21U5QZ2CQECUM|          Four Stars|       AS advertised|
| RL2BBC51H89DH|but has poor insu...|It works as adver...|
|R3RNEPHF3WIRSZ|           Two Stars|it's not worth 22...|
|R38DNT9KML2PF3| Love it for camping|Wonderful! It spi...|
|R2ECMBJM8KNNC8|          Four Stars|Did the job but d...|
|R2F3F92PRN9T7S|Very well satisfied.|Arrived on time a..

In [68]:
# Customers DataFrame
customer_df = appliance_df.groupby("customer_id").agg({"customer_id": "count"}).withColumnRenamed("count(customer_id)", "customer_count")
customer_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   41307088|             1|
|   36030878|             1|
|   52001665|             1|
|    2131939|             1|
|    5901451|             1|
|    4468816|             1|
|   30054413|             1|
|     498070|             1|
|   14632268|             1|
|   41893483|             1|
|   20723844|             1|
|   11955952|             1|
|   19309400|             1|
|   30677319|             1|
|   12651580|             1|
|   13920155|             1|
|     138964|             1|
|   29947344|             1|
|   48607266|             1|
|   15908254|             1|
+-----------+--------------+
only showing top 20 rows



In [69]:
# Vine DataFrame
vine_df = appliance_df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R203HPW78Z7N4K|          5|            0|          0|   N|
|R2EAIGVLEALSP3|          5|            1|          1|   N|
|R1K1CD73HHLILA|          5|            0|          0|   N|
|R2KZBMOFRMYOPO|          5|            1|          1|   N|
| R6BIZOZY6UD01|          5|            0|          0|   N|
|R1MCXZFNF8E7Y0|          1|            0|          0|   N|
|R3EMB3E3ODR6BW|          5|            2|          2|   N|
| RJTONVTTOPJ5S|          5|            0|          0|   N|
|R21U5QZ2CQECUM|          4|            0|          0|   N|
| RL2BBC51H89DH|          4|            0|          0|   N|
|R3RNEPHF3WIRSZ|          2|            0|          0|   N|
|R38DNT9KML2PF3|          5|            0|          0|   N|
|R2ECMBJM8KNNC8|          4|            0|          0|   N|
|R2F3F92PRN9T7S|          5|            

# Push to AWS RDS instance.

In [74]:
mode = "append"
jdbc_url="jdbc:postgresql://big-data-challenge.chjc2dlaerw8.us-east-1.rds.amazonaws.com:5432/bigdatahomework"
config = {"user":"Tennrover", "password": "malone69", "driver":"org.postgresql.Driver"}

In [ ]:
# Write review_id_df to table in RDS
review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [ ]:

# Write review_id_df to table in RDS
review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [ ]:
# Write products_df to table in RDS
prod_df.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [ ]:
# Write customers_df to table in RDS
customer_df.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [ ]:
# Write vine_df to table in RDS
vine_df.write.jdbc(url=jdbc_url, table='vines', mode=mode, properties=config)